In [ ]:
{ "cells": [  {   "cell_type": "markdown",   "id": "course-header",   "metadata": {},   "source": [    "# Module 1: Agent Foundations\n",    "*Building Your First Autonomous AI Agent*\n",    "\n",    "**Learning Objectives:**\n",    "- Understand the core components that make an AI system \"agentic\"\n",    "- Implement the ReAct (Reasoning + Acting) pattern\n",    "- Build a research assistant agent that can search and synthesize information\n",    "\n",    "**Duration:** 45 minutes\n",    "\n",    "---"   ]  },  {   "cell_type": "markdown",   "id": "introduction",   "metadata": {},   "source": [    "## 🎯 What Makes an AI System \"Agentic\"?\n",    "\n",    "Traditional AI systems follow a simple pattern:\n",    "```\n",    "Input → Process → Output\n",    "```\n",    "\n",    "**Agentic AI systems** are fundamentally different. They exhibit **agency** - the ability to:\n",    "- **Perceive** their environment and understand context\n",    "- **Reason** about goals and plan actions to achieve them  \n",    "- **Act** autonomously in their environment\n",    "- **Learn** from feedback and adapt their behavior\n",    "\n",    "This creates a continuous cycle:\n",    "```\n",    "Goal → Plan → Act → Observe → Reflect → Replan → Act → ...\n",    "```\n",    "\n",    "### Key Characteristics of Agentic Systems:\n",    "1. **Autonomy:** Can operate without constant human guidance\n",    "2. **Goal-directed:** Work towards specific objectives\n",    "3. **Adaptive:** Modify behavior based on results\n",    "4. **Interactive:** Can use tools and interact with environments\n",    "5. **Persistent:** Maintain context across multiple interactions"   ]  },  {   "cell_type": "markdown",   "id": "setup",   "metadata": {},   "source": [    "## 🛠️ Environment Setup\n",    "\n",    "Let's start by setting up our development environment with the necessary libraries and helper functions."   ]  },  {   "cell_type": "code",   "execution_count": null,   "id": "imports",   "metadata": {},   "outputs": [],   "source": [    "# Essential imports for building agents\n",    "import openai\n",    "import requests\n",    "import json\n",    "import time\n",    "import re\n",    "from typing import List, Dict, Any, Optional\n",    "from dataclasses import dataclass\n",    "from datetime import datetime\n",    "import os\n",    "from dotenv import load_dotenv\n",    "\n",    "# Load environment variables (API keys, etc.)\n",    "load_dotenv()\n",    "\n",    "# Helper function for pretty printing\n",    "def print_section(title: str, content: str):\n",    "    \"\"\"Helper function to format output sections clearly\"\"\"\n",    "    print(f\"\\n{'='*50}\")\n",    "    print(f\"🤖 {title}\")\n",    "    print(f\"{'='*50}\")\n",    "    print(content)\n",    "    print(f\"{'='*50}\\n\")\n",    "\n",    "print(\"✅ Environment setup complete!\")"   ]  },  {   "cell_type": "markdown",   "id": "agent-anatomy",   "metadata": {},   "source": [    "## 🏗️ Agent Anatomy: The Four Core Components\n",    "\n",    "Every agent consists of four essential components working together:\n",    "\n",    "### 1. **Brain (Language Model)**\n",    "The reasoning engine that processes information and makes decisions\n",    "\n",    "### 2. **Memory System** \n",    "Stores conversation history, learned experiences, and context\n",    "\n",    "### 3. **Tool Arsenal**\n",    "External capabilities the agent can invoke (APIs, databases, calculators, etc.)\n",    "\n",    "### 4. **Planning System**\n",    "Coordinates reasoning and action selection to achieve goals\n",    "\n",    "Let's implement these components step by step:"   ]  },  {   "cell_type": "code",   "execution_count": null,   "id": "agent-base-class",   "metadata": {},   "outputs": [],   "source": [    "# Base Agent class that we'll build upon\n",    "@dataclass\n",    "class AgentAction:\n",    "    \"\"\"Represents an action the agent wants to take\"\"\"\n",    "    tool_name: str\n",    "    tool_input: str\n",    "    reasoning: str\n",    "\n",    "@dataclass\n",    "class AgentObservation:\n",    "    \"\"\"Represents the result of an action\"\"\"\n",    "    content: str\n",    "    success: bool\n",    "    timestamp: datetime\n",    "\n",    "class BasicAgent:\n",    "    \"\"\"A foundational agent class implementing the core components\"\"\"\n",    "    \n",    "    def __init__(self, name: str, api_key: str):\n",    "        # Component 1: Brain (Language Model)\n",    "        self.name = name\n",    "        self.client = openai.OpenAI(api_key=api_key)\n",    "        \n",    "        # Component 2: Memory System\n",    "        self.conversation_memory = []\n",    "        self.experience_memory = []\n",    "        \n",    "        # Component 3: Tool Arsenal (we'll add tools here)\n",    "        self.tools = {}\n",    "        \n",    "        # Component 4: Planning System (tracks current goal and progress)\n",    "        self.current_goal = None\n",    "        self.plan_steps = []\n",    "        \n",    "    def add_tool(self, tool_name: str, tool_function, description: str):\n",    "        \"\"\"Add a new tool to the agent's arsenal\"\"\"\n",    "        self.tools[tool_name] = {\n",    "            'function': tool_function,\n",    "            'description': description\n",    "        }\n",    "        print(f\"✅ Added tool: {tool_name}\")\n",    "    \n",    "    def remember_experience(self, action: AgentAction, observation: AgentObservation):\n",    "        \"\"\"Store experience for future learning\"\"\"\n",    "        experience = {\n",    "            'action': action,\n",    "            'observation': observation,\n",    "            'timestamp': datetime.now()\n",    "        }\n",    "        self.experience_memory.append(experience)\n",    "\n",    "# Let's create our first agent instance!\n",    "# Note: In a real environment, you'd use your actual OpenAI API key\n",    "research_agent = BasicAgent(\n",    "    name=\"ResearchBot\", \n",    "    api_key=os.getenv(\"OPENAI_API_KEY\", \"your-api-key-here\")\n",    ")\n",    "\n",    "print(f\"🎉 Created agent: {research_agent.name}\")\n",    "print(f\"📝 Memory initialized: {len(research_agent.conversation_memory)} conversations\")\n",    "print(f\"🛠️ Tools available: {len(research_agent.tools)} tools\")"   ]  },  {   "cell_type": "markdown",   "id": "tools-section",   "metadata": {},   "source": [    "## 🔧 Building the Tool Arsenal\n",    "\n",    "Tools are what make agents powerful! They extend the agent's capabilities beyond just text generation. Let's implement some essential tools:\n",    "\n",    "### Tool 1: Web Search\n",    "Allows the agent to search for current information"   ]  },  {   "cell_type": "code",   "execution_count": null,   "id": "web-search-tool",   "metadata": {},   "outputs": [],   "source": [    "def web_search_tool(query: str) -> str:\n",    "    \"\"\"\n",    "    Simulates web search functionality.\n",    "    In production, this would integrate with real search APIs like Google, Bing, or Tavily.\n",    "    \"\"\"\n",    "    # For this educational example, we'll simulate search results\n",    "    # In practice, you'd integrate with actual search APIs\n",    "    \n",    "    simulated_results = {\n",    "        \"artificial intelligence\": \"AI is transforming industries through machine learning, automation, and intelligent decision-making systems.\",\n",    "        \"climate change\": \"Climate change refers to global warming and environmental shifts caused by greenhouse gas emissions.\",\n",    "        \"quantum computing\": \"Quantum computing uses quantum mechanics to process information exponentially faster than classical computers.\",\n",    "        \"renewable energy\": \"Solar, wind, and hydroelectric power are leading renewable energy sources for sustainable development.\"\n",    "    }\n",    "    \n",    "    # Simple keyword matching for demonstration\n",    "    for keyword, result in simulated_results.items():\n",    "        if keyword.lower() in query.lower():\n",    "            return f\"Search results for '{query}': {result}\"\n",    "    \n",    "    return f\"Search results for '{query}': Found general information about this topic.\"\n",    "\n",    "# Add the web search tool to our agent\n",    "research_agent.add_tool(\n",    "    tool_name=\"web_search\",\n",    "    tool_function=web_search_tool,\n",    "    description=\"Search the web for current information on any topic\"\n",    ")"   ]  },  {   "cell_type": "markdown",   "id": "calculator-tool",   "metadata": {},   "source": [    "### Tool 2: Calculator\n",    "Enables precise mathematical calculations"   ]  },  {   "cell_type": "code",   "execution_count": null,   "id": "calculator-implementation",   "metadata": {},   "outputs": [],   "source": [    "def calculator_tool(expression: str) -> str:\n",    "    \"\"\"\n",    "    Safely evaluates mathematical expressions.\n",    "    Uses eval() with safety restrictions for educational purposes.\n",    "    \"\"\"\n",    "    try:\n",    "        # Basic safety: only allow mathematical operations\n",    "        allowed_chars = set('0123456789+-*/().^ ')\n",    "        if not all(c in allowed_chars for c in expression):\n",    "            return \"Error: Only basic mathematical operations are allowed\"\n",    "        \n",    "        # Replace ^ with ** for Python exponentiation\n",    "        safe_expression = expression.replace('^', '**')\n",    "        result = eval(safe_expression)\n",    "        return str(result)\n",    "    except Exception as e:\n",    "        return f\"Calculation error: {str(e)}\"\n",    "\n",    "# Add calculator tool to our agent\n",    "research_agent.add_tool(\n",    "    tool_name=\"calculator\",\n",    "    tool_function=calculator_tool,\n",    "    description=\"Perform mathematical calculations with basic operations\"\n",    ")\n",    "\n",    "# Test the calculator\n",    "test_calc = calculator_tool(\"(10 + 5) * 2\")\n",    "print(f\"🧮 Calculator test: (10 + 5) * 2 = {test_calc}\")"   ]  },  {   "cell_type": "markdown",   "id": "note-tool",   "metadata": {},   "source": [    "### Tool 3: Note Taking\n",    "Allows the agent to save important information"   ]  },  {   "cell_type": "code",   "execution_count": null,   "id": "note-taking-tool",   "metadata": {},   "outputs": [],   "source": [    "def note_taking_tool(note_content: str) -> str:\n",    "    \"\"\"\n",    "    Saves important information to the agent's notes.\n",    "    \"\"\"\n",    "    # Initialize notes storage if it doesn't exist\n",    "    if not hasattr(research_agent, 'notes'):\n",    "        research_agent.notes = []\n",    "    \n",    "    # Add timestamped note\n",    "    note = {\n",    "        'content': note_content,\n",    "        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')\n",    "    }\n",    "    research_agent.notes.append(note)\n",    "    \n",    "    return f\"Note saved: '{note_content[:50]}{'...' if len(note_content) > 50 else ''}'\"\n",    "\n",    "# Add note-taking tool\n",    "research_agent.add_tool(\n",    "    tool_name=\"take_note\",\n",    "    tool_function=note_taking_tool,\n",    "    description=\"Save important information to notes for later reference\"\n",    ")\n",    "\n",    "print(f\"🛠️ Agent now has {len(research_agent.tools)} tools available!\")"   ]  },  {   "cell_type": "markdown",   "id": "react-pattern",   "metadata": {},   "source": [    "## 🧠 The ReAct Pattern: Reasoning + Acting\n",    "\n",    "The **ReAct pattern** is a fundamental approach for agentic systems. It alternates between:\n",    "\n",    "1. **Thought**: Reasoning about what to do next\n",    "2. **Action**: Taking a specific action (using a tool)\n",    "3. **Observation**: Observing the result of the action\n",    "4. **Reflection**: Learning from the outcome\n",    "\n",    "This cycle continues until the agent achieves its goal or determines it cannot proceed.\n",    "\n",    "### ReAct Cycle Visualization:\n",    "```\n",    "Goal: \"Research renewable energy trends\"\n",    "↓\n",    "Thought: \"I need to search for recent information\"\n",    "↓  \n",    "Action: web_search(\"renewable energy trends 2024\")\n",    "↓\n",    "Observation: \"Solar and wind energy adoption increased 25%...\"\n",    "↓\n",    "Thought: \"Let me calculate the growth rate\"\n",    "↓\n",    "Action: calculator(\"25 / 100\")\n",    "↓\n",    "Observation: \"0.25\"\n",    "↓\n",    "Thought: \"I should save this key finding\"\n",    "↓\n",    "Action: take_note(\"Renewable energy grew 25% in 2024\")\n",    "↓\n",    "Final Answer: [Comprehensive response]\n",    "```"   ]  },  {   "cell_type": "code",   "execution_count": null,   "id": "react-implementation",   "metadata": {},   "outputs": [],   "source": [    "def execute_react_cycle(agent: BasicAgent, user_goal: str, max_iterations: int = 5) -> str:\n",    "    \"\"\"\n",    "    Implements the ReAct (Reasoning + Acting) pattern.\n",    "    This is the core reasoning loop that makes an agent \"agentic\".\n",    "    \"\"\"\n",    "    \n",    "    # Build the system prompt that teaches the agent how to use ReAct\n",    "    system_prompt = f\"\"\"\n",    "You are {agent.name}, an autonomous AI agent that can use tools to accomplish tasks.\n",    "\n",    "Available tools:\n",    "\"\"\"\n",    "    \n",    "    # Add tool descriptions to the prompt\n",    "    for tool_name, tool_info in agent.tools.items():\n",    "        system_prompt += f\"- {tool_name}: {tool_info['description']}\\n\"\n",    "    \n",    "    system_prompt += \"\"\"\n",    "Use the ReAct pattern to accomplish your goal:\n",    "\n",    "Thought: [Reason about what you need to do next]\n",    "Action: [tool_name]\n",    "Action Input: [input for the tool]\n",    "Observation: [You will see the tool result here]\n",    "\n",    "Continue this cycle until you can provide a Final Answer.\n",    "When you have enough information, respond with:\n",    "Final Answer: [Your complete response to the user]\n",    "\n",    "Important: Always start with a Thought about your approach.\n",    "\"\"\"\n",    "    \n",    "    # Initialize conversation with the user's goal\n",    "    messages = [\n",    "        {\"role\": \"system\", \"content\": system_prompt},\n",    "        {\"role\": \"user\", \"content\": user_goal}\n",    "    ]\n",    "    \n",    "    print_section(\"STARTING REACT CYCLE\", f\"Goal: {user_goal}\")\n",    "    \n",    "    for iteration in range(max_iterations):\n",    "        print(f\"\\n--- Iteration {iteration + 1} ---\")\n",    "        \n",    "        # Get response from the language model\n",    "        try:\n",    "            response = agent.client.chat.completions.create(\n",    "                model=\"gpt-4\",\n",    "                messages=messages,\n",    "                temperature=0.1,\n",    "                max_tokens=1000\n",    "            )\n",    "            \n",    "            assistant_message = response.choices[0].message.content\n",    "            print(f\"🤖 Agent Response:\\n{assistant_message}\")\n",    "            \n",    "            messages.append({\"role\": \"assistant\", \"content\": assistant_message})\n",    "            \n",    "        except Exception as e:\n",    "            print(f\"❌ Error calling language model: {e}\")\n",    "            return \"Error: Could not get response from language model\"\n",    "        \n",    "        # Check if we have a final answer\n",    "        if \"Final Answer:\" in assistant_message:\n",    "            final_answer = assistant_message.split(\"Final Answer:\")[-1].strip()\n",    "            print_section(\"TASK COMPLETED\", final_answer)\n",    "            return final_answer\n",    "        \n",    "        # Parse and execute action\n",    "        action_match = re.search(r\"Action: (\\w+)\", assistant_message)\n",    "        action_input_match = re.search(r\"Action Input: (.+)\", assistant_message)\n",    "        \n",    "        if action_match and action_input_match:\n",    "            tool_name = action_match.group(1)\n",    "            tool_input = action_input_match.group(1).strip()\n",    "            \n",    "            print(f\"🔧 Executing: {tool_name}({tool_input})\")\n",    "            \n",    "            # Execute the tool\n",    "            if tool_name in agent.tools:\n",    "                try:\n",    "                    result = agent.tools[tool_name]['function'](tool_input)\n",    "                    observation = f\"Observation: {result}\"\n",    "                    print(f\"👀 {observation}\")\n",    "                    \n",    "                    # Add observation to conversation\n",    "                    messages.append({\"role\": \"user\", \"content\": observation})\n",    "                    \n",    "                    # Remember this experience\n",    "                    action = AgentAction(\n",    "                        tool_name=tool_name,\n",    "                        tool_input=tool_input,\n",    "                        reasoning=assistant_message\n",    "                    )\n",    "                    obs = AgentObservation(\n",    "                        content=result,\n",    "                        success=True,\n",    "                        timestamp=datetime.now()\n",    "                    )\n",    "                    agent.remember_experience(action, obs)\n",    "                    \n",    "                except Exception as e:\n",    "                    error_msg = f\"Tool execution failed: {str(e)}\"\n",    "                    print(f\"❌ {error_msg}\")\n",    "                    messages.append({\"role\": \"user\", \"content\": f\"Observation: {error_msg}\"})\n",    "            else:\n",    "                error_msg = f\"Tool '{tool_name}' not found. Available tools: {list(agent.tools.keys())}\"\n",    "                print(f\"❌ {error_msg}\")\n",    "                messages.append({\"role\": \"user\", \"content\": f\"Observation: {error_msg}\"})\n",    "        else:\n",    "            print(\"⚠️ Could not parse action from response\")\n",    "            messages.append({\"role\": \"user\", \"content\": \"Observation: Could not parse action. Please use the format 'Action: tool_name' and 'Action Input: input'\"})\n",    "    \n",    "    return \"Maximum iterations reached without completing the task\"\n",    "\n",    "print(\"✅ ReAct cycle implementation ready!\")"   ]  },  {   "cell_type": "markdown",   "id": "first-exercise",   "metadata": {},   "source": [    "## 🎯 Hands-On Exercise: Your First Agentic Task\n",    "\n",    "Now it's time to see your agent in action! We'll give it a research task that requires multiple steps and tool usage.\n",    "\n",    "**Task:** \"Research the environmental impact of renewable energy and calculate the potential CO2 reduction if 50% of current fossil fuel energy was replaced by renewables.\"\n",    "\n",    "Watch how the agent:\n",    "1. **Reasons** about what information it needs\n",    "2. **Searches** for relevant data\n",    "3. **Calculates** specific numbers\n",    "4. **Synthesizes** the information into a comprehensive answer"   ]  },  {   "cell_type": "code",   "execution_count": null,   "id": "exercise-execution",   "metadata": {},   "outputs": [],   "source": [    "# Let's run our first agentic task!\n",    "research_task = \"\"\"\n",    "Research the environmental impact of renewable energy and calculate the potential CO2 reduction \n",    "if 50% of current fossil fuel energy was replaced by renewables. \n",    "Please provide specific numbers and save the key findings to your notes.\n",    "\"\"\"\n",    "\n",    "# Execute the task using our ReAct implementation\n",    "result = execute_react_cycle(research_agent, research_task)\n",    "\n",    "print(\"\\n\" + \"=\"*60)\n",    "print(\"📊 AGENT LEARNING SUMMARY\")\n",    "print(\"=\"*60)\n",    "print(f\"Experiences gained: {len(research_agent.experience_memory)}\")\n",    "if hasattr(research_agent, 'notes'):\n",    "    print(f\"Notes taken: {len(research_agent.notes)}\")\n",    "    for i, note in enumerate(research_agent.notes):\n",    "        print(f\"  {i+1}. {note['content'][:100]}...\")"   ]  },  {   "cell_type": "markdown",   "id": "guided-exercise",   "metadata": {},   "source": [    "## 💡 Guided Exercise: Build Your Own Tool\n",    "\n",    "Now it's your turn! Complete the code below to add a new tool to the agent.\n",    "\n",    "**Your Task:** Implement a \"summarizer\" tool that can create concise summaries of long text."   ]  },  {   "cell_type": "code",   "execution_count": null,   "id": "guided-exercise-code",   "metadata": {},   "outputs": [],   "source": [    "def summarizer_tool(text: str) -> str:\n",    "    \"\"\"\n",    "    Create a concise summary of the input text.\n",    "    \n",    "    YOUR TASK: Complete this function implementation.\n",    "    \n",    "    Hints:\n",    "    - For this exercise, create a simple rule-based summarizer\n",    "    - Extract the first and last sentences\n",    "    - Or find sentences containing key words like \"important\", \"key\", \"significant\"\n",    "    - Return a summary that's roughly 1/3 the length of the original\n",    "    \"\"\"\n",    "    \n",    "    ### START CODE HERE ###\n",    "    \n",    "    # Split text into sentences\n",    "    sentences = # YOUR CODE: Split the text into sentences\n",    "    \n",    "    if len(sentences) <= 2:\n",    "        return text  # Return original if already short\n",    "    \n",    "    # Simple summarization strategy: first sentence + key sentences + last sentence\n",    "    summary_sentences = []\n",    "    \n",    "    # Always include first sentence\n",    "    summary_sentences.append(# YOUR CODE: Add first sentence)\n",    "    \n",    "    # Find sentences with key words\n",    "    key_words = ['important', 'significant', 'key', 'crucial', 'main', 'primary']\n",    "    for sentence in sentences[1:-1]:  # Skip first and last\n",    "        # YOUR CODE: Check if sentence contains any key words\n",    "        if # YOUR CONDITION:\n",    "            summary_sentences.append(sentence)\n",    "    \n",    "    # Always include last sentence if we have room\n",    "    if len(summary_sentences) < len(sentences) // 2:\n",    "        summary_sentences.append(# YOUR CODE: Add last sentence)\n",    "    \n",    "    # Join and return\n",    "    return # YOUR CODE: Join summary sentences\n",    "    \n",    "    ### END CODE HERE ###\n",    "\n",    "# Test your implementation\n",    "test_text = \"\"\"\n",    "Artificial intelligence is transforming many industries today. \n",    "Machine learning algorithms can process vast amounts of data quickly. \n",    "The key benefit is improved decision-making capabilities. \n",    "Companies are investing heavily in AI research and development. \n",    "However, important ethical considerations must be addressed. \n",    "The future of AI looks very promising for solving complex problems.\n",    "\"\"\"\n",    "\n",    "test_summary = summarizer_tool(test_text.strip())\n",    "print(\"📄 Original text:\")\n",    "print(test_text)\n",    "print(\"\\n📝 Your summary:\")\n",    "print(test_summary)\n",    "\n",    "# Add your tool to the agent\n",    "research_agent.add_tool(\n",    "    tool_name=\"summarize\",\n",    "    tool_function=summarizer_tool,\n",    "    description=\"Create a concise summary of long text content\"\n",    ")"   ]  },  {   "cell_type": "markdown",   "id": "solution-reveal",   "metadata": {},   "source": [    "### 🔍 Solution Reveal\n",    "\n",    "Click below to see the solution if you need help:"   ]  },  {   "cell_type": "code",   "execution_count": null,   "id": "solution-code",   "metadata": {},   "outputs": [],   "source": [    "# SOLUTION - Don't peek until you've tried!\n",    "\"\"\"\n",    "def summarizer_tool(text: str) -> str:\n",    "    # Split text into sentences\n",    "    sentences = [s.strip() for s in text.split('.') if s.strip()]\n",    "    \n",    "    if len(sentences) <= 2:\n",    "        return text\n",    "    \n",    "    summary_sentences = []\n",    "    \n",    "    # Always include first sentence\n",    "    summary_sentences.append(sentences[0])\n",    "    \n",    "    # Find sentences with key words\n",    "    key_words = ['important', 'significant', 'key', 'crucial', 'main', 'primary']\n",    "    for sentence in sentences[1:-1]:\n",    "        if any(word in sentence.lower() for word in key_words):\n",    "            summary_sentences.append(sentence)\n",    "    \n",    "    # Always include last sentence if we have room\n",    "    if len(summary_sentences) < len(sentences) // 2:\n",    "        summary_sentences.append(sentences[-1])\n",    "    \n",    "    return '. '.join(summary_sentences) + '.'\n",    "\"\"\"\n",    "\n",    "print(\"💡 Solution revealed above! Compare with your implementation.\")"   ]  },  {   "cell_type": "markdown",   "id": "test-new-tool",   "metadata": {},   "source": [    "## 🧪 Test Your Enhanced Agent\n",    "\n",    "Now let's test your agent with its new summarization capability!"   ]  },  {   "cell_type": "code",   "execution_count": null,   "id": "test-enhanced-agent",   "metadata": {},   "outputs": [],   "source": [    "# Test the enhanced agent with a task that uses the new tool\n",    "enhanced_task = \"\"\"\n",    "Research information about quantum computing, then summarize the key points, \n",    "and save a note about the most important finding.\n",    "\"\"\"\n",    "\n",    "enhanced_result = execute_react_cycle(research_agent, enhanced_task)\n",    "\n",    "print(f\"\\n🎯 Agent now has {len(research_agent.tools)} tools:\")\n",    "for tool_name in research_agent.tools.keys():\n",    "    print(f\"  ✅ {tool_name}\")"   ]  },  {   "cell_type": "markdown",   "id": "knowledge-check",   "metadata": {},   "source": [    "## 📝 Knowledge Check\n",    "\n",    "Test your understanding with these questions:"   ]  },  {   "cell_type": "code",   "execution_count": null,   "id": "knowledge-check-code",   "metadata": {},   "outputs": [],   "source": [    "# Interactive Knowledge Check\n",    "def knowledge_check():\n",    "    questions = [\n",    "        {\n",    "            \"question\": \"What are the four core components of an agentic system?\",\n",    "            \"options\": [\n",    "                \"A) Input, Process, Output, Memory\",\n",    "                \"B) Brain, Memory, Tools, Planning\", \n",    "                \"C) Data, Model, API, Interface\",\n",    "                \"D) Search, Calculate, Summarize, Store\"\n",    "            ],\n",    "            \"correct\": \"B\"\n",    "        },\n",    "        {\n",    "            \"question\": \"What does ReAct stand for?\",\n",    "            \"options\": [\n",    "                \"A) Retrieve and Act\",\n",    "                \"B) Reason and Activate\",\n",    "                \"C) Reasoning and Acting\",\n",    "                \"D) Research and Action\"\n",    "            ],\n",    "            \"correct\": \"C\"\n",    "        },\n",    "        {\n",    "            \"question\": \"What makes an AI system 'agentic'?\",\n",    "            \"options\": [\n",    "                \"A) Using large language models\",\n",    "                \"B) Having a nice user interface\", \n",    "                \"C) Autonomy, goal-direction, and environmental interaction\",\n",    "                \"D) Being very fast and accurate\"\n",    "            ],\n",    "            \"correct\": \"C\"\n",    "        }\n",    "    ]\n",    "    \n",    "    score = 0\n",    "    for i, q in enumerate(questions, 1):\n",    "        print(f\"\\nQuestion {i}: {q['question']}\")\n",    "        for option in q['options']:\n",    "            print(f\"  {option}\")\n",    "        \n",    "        # In a real notebook, you'd use input() here\n",    "        # For this example, we'll show the correct answer\n",    "        print(f\"\\n✅ Correct Answer: {q['correct']}\")\n",    "        \n",    "        # Explanation\n",    "        if q['correct'] == 'B' and i == 1:\n",    "            print(\"💡 The four components work together: Brain (LLM) for reasoning, Memory for context, Tools for capabilities, Planning for coordination.\")\n",    "        elif q['correct'] == 'C' and i == 2:\n",    "            print(\"💡 ReAct alternates between reasoning about what to do and taking actions to achieve goals.\")\n",    "        elif q['correct'] == 'C' and i == 3:\n",    "            print(\"💡 Agentic systems can operate autonomously, work toward goals, and interact with their environment.\")\n",    "    \n",    "    print(\"\\n🎯 Knowledge check complete! Review any concepts you'd like to understand better.\")\n",    "\n",    "knowledge_check()"   ]  },  {   "cell_type": "markdown",   "id": "reflection",   "metadata": {},   "source": [    "## 🎯 Module 1 Summary\n",    "\n",    "Congratulations! You've successfully built your first agentic AI system. Let's reflect on what you've accomplished:\n",    "\n",    "### ✅ What You Built:\n",    "- **Complete Agent Architecture** with all four core components\n",    "- **ReAct Implementation** that can reason and act autonomously\n",    "- **Tool Integration System** with web search, calculator, and note-taking\n",    "- **Memory System** that learns from experience\n",    "- **Custom Tool Development** (your summarizer)\n",    "\n",    "### 🧠 Key Concepts Mastered:\n",    "- **Agency** vs. traditional AI systems\n",    "- **ReAct Pattern** for autonomous reasoning\n",    "- **Tool Integration** for extending capabilities\n",    "- **Memory and Learning** for continuous improvement\n",    "\n",    "### 🚀 Next Steps:\n",    "In Module 2, you'll learn to:\n",    "- Implement advanced memory systems\n",    "- Create agents that learn from conversation history\n",    "- Build context-aware conversational agents\n",    "- Handle long-term memory and knowledge retention\n",    "\n",    "### 💪 Challenge Exercise (Optional):\n",    "Before moving to Module 2, try enhancing your agent with:\n",    "1. A \"weather\" tool that provides current weather information\n",    "2. A \"translator\" tool for basic language translation\n",    "3. A task that requires using all tools in sequence\n",    "\n",    "### 📊 Progress Tracker:"   ]  },  {   "cell_type": "code",   "execution_count": null,   "id": "progress-tracker",   "metadata": {},   "outputs": [],   "source": [    "# Progress Summary\n",    "def module_progress_summary():\n",    "    print(\"📈 MODULE 1 PROGRESS SUMMARY\")\n",    "    print(\"=\" * 40)\n",    "    \n",    "    # Agent capabilities\n",    "    capabilities = [\n",    "        \"✅ Basic agent architecture\",\n",    "        \"✅ ReAct reasoning pattern\", \n",    "        \"✅ Tool integration system\",\n",    "        \"✅ Memory and experience tracking\",\n",    "        \"✅ Custom tool development\"\n",    "    ]\n",    "    \n",    "    print(\"\\n🛠️ Agent Capabilities Built:\")\n",    "    for cap in capabilities:\n",    "        print(f\"  {cap}\")\n",    "    \n",    "    print(f\"\\n📊 Agent Statistics:\")\n",    "    print(f\"  • Tools available: {len(research_agent.tools)}\")\n",    "    print(f\"  • Experiences gained: {len(research_agent.experience_memory)}\")\n",    "    if hasattr(research_agent, 'notes'):\n",    "        print(f\"  • Notes taken: {len(research_agent.notes)}\")\n",    "    \n",    "    print(f\"\\n🎯 Ready for Module 2: Advanced Memory and Learning!\")\n",    "    print(f\"\\n💡 Pro Tip: The agent you built here is the foundation for all advanced patterns we'll explore!\")\n",    "\n",    "module_progress_summary()"   ]  }\n", ],\n", "metadata": {\n",  \"kernelspec\": {\n",   \"display_name\": \"Python 3\",\n",   \"language\": \"python\",\n",   \"name\": \"python3\"\n",  },\n",  \"language_info\": {\n",   \"codemirror_mode\": {\n",    \"name\": \"ipython\",\n",    \"version\": 3\n",   },\n",   \"file_extension\": \".py\",\n",   \"mimetype\": \"text/x-python\",\n",   \"name\": \"python\",\n",   \"nbconvert_exporter\": \"python\",\n",   \"pygments_lexer\": \"ipython3\",\n",   \"version\": \"3.8.0\"\n",  }\n }\n}